In [12]:
import numpy as np
import pandas as pd
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

def str_to_datetime(s):
    split = s.split('-')
    year, month, day = int(split[0]), int(split[1]), int(split[2]) 
    return datetime.datetime(year=year, month=month, day=day)

def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
    first_date = str_to_datetime(first_date_str)
    last_date = str_to_datetime(last_date_str)

    target_date = first_date
  
    dates = []
    X, Y = [], []

    last_time = False
    while True:
        df_subset = dataframe.loc[:target_date].tail(n+1)
    
        if len(df_subset) != n+1:
            print(f'Error: Window of size {n} is too large for date {target_date}')
            return

        values = df_subset['Close'].to_numpy()
        x, y = values[:-1], values[-1]

        dates.append(target_date)
        X.append(x)
        Y.append(y)

        next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
        next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
        next_date_str = next_datetime_str.split('T')[0]
        year_month_day = next_date_str.split('-')
        year, month, day = year_month_day
        next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))
    
        if last_time:
            break
    
        target_date = next_date

        if target_date == last_date:
            last_time = True
    
    ret_df = pd.DataFrame({})
    ret_df['Target Date'] = dates
  
    X = np.array(X)
    for i in range(0, n):
        X[:, i]
        ret_df[f'Target-{n-i}'] = X[:, i]
  
    ret_df['Target'] = Y

    return ret_df

def windowed_df_to_date_X_y(windowed_dataframe, model):
    df_as_np = windowed_dataframe.to_numpy()
    dates = df_as_np[:, 0]
    mat1 = df_as_np[:, 1:-1] 
    X = mat1.reshape((len(dates), mat1.shape[1], 1)) 
    y = df_as_np[:, -1]
    return dates, X.astype(np.float32), y.astype(np.float32) 

def predict_stock_price_next_day(file_path, first_date_str, last_date_str, n=3, epochs=100):
    # Load data from CSV
    df = pd.read_csv(file_path) 
    df = df[['Date', 'Close']] 

    # Convert 'Date' column to datetime object and set as index
    df['Date'] = df['Date'].apply(str_to_datetime)
    df.index = df.pop('Date')

    # Define the model architecture
    model = Sequential([
        layers.Input((n, 1)),
        layers.LSTM(64),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])

    # Compile the model
    model.compile(loss='mse', 
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['mean_absolute_error'])

    # Create windowed DataFrame
    windowed_df = df_to_windowed_df(df, first_date_str, last_date_str, n)

    # Get date, X, y arrays
    dates, X, y = windowed_df_to_date_X_y(windowed_df, model)

    # Train the model
    model.fit(X, y, epochs=epochs)

    # Predict the stock price for the following day
    last_date = str_to_datetime(last_date_str)
    next_day_date = last_date + datetime.timedelta(days=1)
    next_day_input = df.loc[last_date-n+1:last_date, 'Close'].values.reshape((1, n, 1))

    next_day_prediction = model.predict(next_day_input)

    return next_day_date, next_day_prediction

    # Train the model
    model.fit(X, y, epochs=epochs)

    # Predict the stock price for the following day
    last_date = str_to_datetime(last_date_str)
    next_day_date = last_date + datetime.timedelta(days=1)

    next_day_input_array = df.loc[last_date - datetime.timedelta(days=n-1):last_date, 'Close'].values
    print("Size of next_day_input_array:", len(next_day_input_array))
    next_day_input = next_day_input_array.reshape((1, n, 1))
    
    next_day_input = df.loc[last_date - datetime.timedelta(days=n-1):last_date, 'Close'].values.reshape((1, n, 1)

    next_day_prediction = model.predict(next_day_input)

    return next_day_date, next_day_prediction

# Example usage:
file_path = input("Please input a Yahoo Finance CSV file path for a stock of your choice: ")
first_date_str = input("Please input the first date in the format 'YYYY-MM-DD': ")
last_date_str = input("Please input the last date in the format 'YYYY-MM-DD': ")

next_day_date, next_day_prediction = predict_stock_price_next_day(file_path, first_date_str, last_date_str)

print(f"The predicted stock price for {next_day_date.strftime('%Y-%m-%d')} is: {next_day_prediction[0][0]}")


Please input a Yahoo Finance CSV file path for a stock of your choice:  ~/StockData/AAPL.csv
Please input the first date in the format 'YYYY-MM-DD':  2024-02-03
Please input the last date in the format 'YYYY-MM-DD':  2024-02-06


Epoch 1/100
1/1 [==============================] - 1s 900ms/step - loss: 35300.4141 - mean_absolute_error: 187.8761
Epoch 2/100
1/1 [==============================] - 0s 5ms/step - loss: 35268.9180 - mean_absolute_error: 187.7923
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 35235.1406 - mean_absolute_error: 187.7023
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 35197.9297 - mean_absolute_error: 187.6032
Epoch 5/100
1/1 [==============================] - 0s 4ms/step - loss: 35154.1484 - mean_absolute_error: 187.4865
Epoch 6/100
1/1 [==============================] - 0s 5ms/step - loss: 35111.2891 - mean_absolute_error: 187.3721
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 35076.0156 - mean_absolute_error: 187.2780
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 35043.6484 - mean_absolute_error: 187.1915
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 35025.0547 - me

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'int'